In [ ]:
import torch
import numpy as np

import librosa
import pyttsx3
from TTS.api import TTS

from IPython.display import Audio, clear_output

from so_vits_svc_fork.inference.core import Svc
from so_vits_svc_fork.utils import get_optimal_device

In [ ]:
def text_to_speech_old(text, sr_target):
    engine = pyttsx3.init()  # Initialize the text-to-speech engine

    # Set the properties for the engine
    engine.setProperty('rate', 125)     # words spoken per minute
    engine.setProperty('volume', 1.0)   # volume level between 0 and 1

    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)  # set the voice to female

    # Use pyttsx3 to generate audio and save it
    engine.save_to_file(text, 'audio.wav')
    engine.runAndWait()

    # Load audio from the byte array using soundfile
    audio, _ = librosa.load('audio.wav', sr=sr_target)

    return audio

def text_to_speech(text, sr_target):
    # List available 🐸TTS models and choose the first one
    # print(TTS.list_models())
    model_name = TTS.list_models()[0]
    # model_name = 'tts_models/en/ljspeech/glow-tts'
    # Init TTS
    tts = TTS(model_name, progress_bar=True, gpu=True)
    # Text to speech to a file
    print(tts.speakers)
    print(tts.languages)
    tts.tts_to_file(text=text, speaker=tts.speakers[3], language=tts.languages[0], file_path="audio.wav", emotion="Angry")
    #multi speaker model requires these speaker=tts.speakers[0], language=tts.languages[0],
    #emotions
    #Neutral Dull Happy Sad Surprise Angry


    # Load audio from the byte array using soundfile
    audio, _ = librosa.load('audio.wav', sr=sr_target)

    return audio



audio = text_to_speech("testing text to speech", 44100)
Audio(data=audio, rate=44100)

In [ ]:
model_path = ""
config_path = ""
cluster_model_path = ""
device: str | torch.device = get_optimal_device()

svc_model = Svc(
        net_g_path=model_path,
        config_path=config_path,
        cluster_model_path=cluster_model_path
        if cluster_model_path
        else None,
        device=device,
    )

print("Warming up the model...")
svc_model.infer(
    speaker=0,
    transpose=0,
    auto_predict_f0=False,
    cluster_infer_ratio=0,
    noise_scale=0.4,
    f0_method="dio", #"crepe", "crepe-tiny", "parselmouth", "dio", "harvest"
    audio=np.zeros(svc_model.target_sample, dtype=np.float32),
)

# audio, _ = librosa.load("", sr=svc_model.target_sample)

# audio = svc_model.infer_silence(
#                 audio.astype(np.float32),
#                 speaker=0,
#                 transpose=0,
#                 auto_predict_f0=True,
#                 cluster_infer_ratio=0,
#                 noise_scale=0.4,
#                 f0_method="dio", #"crepe", "crepe-tiny", "parselmouth", "dio", "harvest"
#                 db_thresh=-40,
#                 pad_seconds=0.5,
#                 chunk_seconds=0.5,
#                 absolute_thresh=False,
#                 max_chunk_seconds=40,
#             )

# Clear the output
clear_output(wait=False)         

In [ ]:
audio = text_to_speech("\
                        🐸TTS is tested on Ubuntu 18.04 with python >= 3.7, < 3.11..\
If you are only interested in synthesizing speech with the released 🐸TTS models, installing from PyPI is the easiest option.\
                       ", svc_model.target_sample)


audio = svc_model.infer_silence(
                audio.astype(np.float32),
                speaker=0,
                transpose=0,
                auto_predict_f0=True,
                cluster_infer_ratio=0,
                noise_scale=0.1,
                f0_method="dio", #"crepe", "crepe-tiny", "parselmouth", "dio", "harvest"
                db_thresh=-100,
                pad_seconds=1,
                chunk_seconds=0.5,
                absolute_thresh=False,
                max_chunk_seconds=40,
            )

# Clear the output
clear_output(wait=True)
Audio(data=audio, rate=svc_model.target_sample)